In [91]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
import re
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import warnings
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix
from imblearn.over_sampling import SMOTE, ADASYN
warnings.filterwarnings('ignore')
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_selection import SelectKBest, chi2

In [92]:
df = pd.read_csv("/content/Book.xlsx (2).csv")
df.head()

,sentence,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,তোদের জন্মের দোষ আছে।,NaN,NaN,NaN,NaN,NaN
1,দুই বউ নিয়ে মজা করছে লুচ্চা মুস্তাক।,NaN,NaN,NaN,NaN,NaN
2,"ওর সমস্ত পরিবারটা নির্লজ্জ,,",NaN,NaN,NaN,NaN,NaN
3,এদেরকে পাবনা পাঠাও,NaN,NaN,NaN,NaN,NaN
4,লজ্জা নেই বলে এই রকম,NaN,NaN,NaN,NaN,NaN


In [93]:
df.drop(columns=['Unnamed: 1','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5'], inplace=True)

In [94]:
df

,sentence
0,তোদের জন্মের দোষ আছে।
1,দুই বউ নিয়ে মজা করছে লুচ্চা মুস্তাক।
2,"ওর সমস্ত পরিবারটা নির্লজ্জ,,"
3,এদেরকে পাবনা পাঠাও
4,লজ্জা নেই বলে এই রকম
...,...
494,মহিলা তুমিও লোভি লায়লার টাকায় তোমার ছেলে ওতুমি...
495,গচার বেরাম আছে
496,"আপনি অপরাধী,, এমন কু সন্তান জন্ম দিলেন কু সন্ত..."
497,বাজে মহিলা


In [95]:
pip install emoji pandas


In [96]:
def punctuations_remover(my_str):
    if isinstance(my_str, str):
        # Define the set of punctuation characters to be removed
        punctuations = '''````£|¢|Ñ+-*/=EROero৳০১২৩৪৫৬৭৮৯012–34567•89।!()-[]{};:'"“\’,<>./?@#$%^&*_~‘—॥”‰🤣⚽️✌�￰৷￰'''
        # Use a list comprehension for more efficient character filtering
        no_punct = ''.join([char for char in my_str if char not in punctuations])
        # Replace newline characters with space
        no_punct = no_punct.replace("\n", ' ')
        # Replace multiple spaces with a single space
        no_punct = re.sub(' +', ' ', no_punct)
        return no_punct
    else:
        return ''


In [97]:
df['sentence'] = df['sentence'].apply(punctuations_remover)

In [98]:
df

,sentence
0,তোদের জন্মের দোষ আছে
1,দুই বউ নিয়ে মজা করছে লুচ্চা মুস্তাক
2,ওর সমস্ত পরিবারটা নির্লজ্জ
3,এদেরকে পাবনা পাঠাও
4,লজ্জা নেই বলে এই রকম
...,...
494,মহিলা তুমিও লোভি লায়লার টাকায় তোমার ছেলে ওতুমি...
495,গচার বেরাম আছে
496,আপনি অপরাধী এমন কু সন্তান জন্ম দিলেন কু সন্তান...
497,বাজে মহিলা


In [99]:
# Define function for text cleaning
def text_cleaner(text):
    # Remove non-Bengali characters and punctuation
    cleaned_text = re.sub(r'[^\u0980-\u09FF\s]', '', text)
    # Convert to lowercase
    cleaned_text = cleaned_text.lower()
    return cleaned_text

# Apply text cleaning
df['sentence'] = df['sentence'].apply(text_cleaner)

In [100]:
df

,sentence
0,তোদের জন্মের দোষ আছে
1,দুই বউ নিয়ে মজা করছে লুচ্চা মুস্তাক
2,ওর সমস্ত পরিবারটা নির্লজ্জ
3,এদেরকে পাবনা পাঠাও
4,লজ্জা নেই বলে এই রকম
...,...
494,মহিলা তুমিও লোভি লায়লার টাকায় তোমার ছেলে ওতুমি...
495,গচার বেরাম আছে
496,আপনি অপরাধী এমন কু সন্তান জন্ম দিলেন কু সন্তান...
497,বাজে মহিলা


In [101]:
import pandas as pd

In [102]:
!pip install googletrans==4.0.0-rc1

In [103]:
from googletrans import Translator

translator = Translator()

def translate_text(text):
    try:
        translation = translator.translate(text, src='bn', dest='en')
        return translation.text
    except Exception as e:
        print(f"Error in translation: {e}")
        return text

In [104]:
subset_df = df.head(50)

In [105]:
subset_df

,sentence
0,তোদের জন্মের দোষ আছে
1,দুই বউ নিয়ে মজা করছে লুচ্চা মুস্তাক
2,ওর সমস্ত পরিবারটা নির্লজ্জ
3,এদেরকে পাবনা পাঠাও
4,লজ্জা নেই বলে এই রকম
5,বিশ্ব বেহায়া
6,এরা সফল কারণ এদের লজ্জা শরম বলতে কিছুই নেই
7,নিলজ্জাহি মহিলা
8,বুড়ো বয়সে প্রেম রতি ধরেছে
9,লজ্জা সরম না থাকলে যা হয়


In [106]:
subset_df['sentence_en'] = subset_df['sentence'].apply(translate_text)

In [107]:
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Lowercase
    text = text.lower()
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove punctuations and numbers
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\d', '', text)
    # Remove stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [108]:
subset_df['sentence_clean'] = subset_df['sentence_en'].apply(preprocess_text)

In [109]:
subset_df

,sentence,sentence_en,sentence_clean
0,তোদের জন্মের দোষ আছে,You have the fault of your birth,fault birth
1,দুই বউ নিয়ে মজা করছে লুচ্চা মুস্তাক,Lucha Mustak is joking with two wives,lucha mustak joking two wives
2,ওর সমস্ত পরিবারটা নির্লজ্জ,All of his family is shameless,family shameless
3,এদেরকে পাবনা পাঠাও,Send them to Pabna,send pabna
4,লজ্জা নেই বলে এই রকম,Like this,like
5,বিশ্ব বেহায়া,The world is pernicious,world pernicious
6,এরা সফল কারণ এদের লজ্জা শরম বলতে কিছুই নেই,They are successful because they have nothing ...,successful nothing say shame
7,নিলজ্জাহি মহিলা,Niljahi,niljahi
8,বুড়ো বয়সে প্রেম রতি ধরেছে,The old man holds love rhythm,old man holds love rhythm
9,লজ্জা সরম না থাকলে যা হয়,If the shame is not a simple,shame simple


In [110]:
pip install transformers

In [111]:
from transformers import pipeline

# Load pre-trained hate speech detection pipeline
classifier = pipeline("text-classification", model="unitary/toxic-bert")

def detect_hate_speech(text):
    result = classifier(text)
    return result

In [112]:
X=subset_df['sentence']

In [113]:
X

0                                  তোদের জন্মের দোষ আছে
1                   দুই বউ নিয়ে মজা করছে লুচ্চা মুস্তাক
2                            ওর সমস্ত পরিবারটা নির্লজ্জ
3                                    এদেরকে পাবনা পাঠাও
4                                  লজ্জা নেই বলে এই রকম
5                                         বিশ্ব বেহায়া
6            এরা সফল কারণ এদের লজ্জা শরম বলতে কিছুই নেই
7                                       নিলজ্জাহি মহিলা
8                           বুড়ো বয়সে প্রেম রতি ধরেছে
9                              লজ্জা সরম না থাকলে যা হয়
10    তাদের প্রবলেম থাকবো না তুই তো সমাজ নষ্ট করতে ত...
11                                        ইবলিশের বংশধর
12               বুড়ো বেটা কি বলে রে বুড়া বেটা কি বলে
13                                          বুড়ো পাগলা
14                             ভাব দেখে শরীর জ্বলে যায়
15    নরসিংদির পোলাকোন দিক দিয়ে এই বুড়া বেডাকে পোলা ...
16                                    গরিবের মিয়া খলিফা
17                          অল্প বয়সে পেকে গেলে

In [114]:
Y=subset_df['sentence_clean']

In [115]:
Y

0                                           fault birth
1                         lucha mustak joking two wives
2                                      family shameless
3                                            send pabna
4                                                  like
5                                      world pernicious
6                          successful nothing say shame
7                                               niljahi
8                             old man holds love rhythm
9                                          shame simple
10          problem understand anything destroy society
11                                                iblis
12                            old beta ray old beta say
13                                            old crazy
14                                          body burned
15      narsingdi pole direction old bed looks like bod
16                                          mia khalifa
17                                              

In [116]:
import pandas as pd

def process_dataset(dataset):
    translated_texts = [translate_text(text) for text in dataset]
    results = [detect_hate_speech(text) for text in translated_texts]
    return results

In [117]:
results = process_dataset(X)

In [118]:
for i, (X, results) in enumerate(zip(X, results)):
    print(f"Original Bangla Text: {X}")
    print(f"Translated English Text: {translate_text(X)}")
    print(f"Hate Speech Detection Result: {results}")
    print()

Original Bangla Text: তোদের জন্মের দোষ আছে
Translated English Text: You have the fault of your birth
Hate Speech Detection Result: [{'label': 'toxic', 'score': 0.4925139546394348}]

Original Bangla Text: দুই বউ নিয়ে মজা করছে লুচ্চা মুস্তাক
Translated English Text: Lucha Mustak is joking with two wives
Hate Speech Detection Result: [{'label': 'toxic', 'score': 0.00716389762237668}]

Original Bangla Text: ওর সমস্ত পরিবারটা নির্লজ্জ
Translated English Text: All of his family is shameless
Hate Speech Detection Result: [{'label': 'toxic', 'score': 0.48732465505599976}]

Original Bangla Text: এদেরকে পাবনা পাঠাও
Translated English Text: Send them to Pabna
Hate Speech Detection Result: [{'label': 'toxic', 'score': 0.022534504532814026}]

Original Bangla Text: লজ্জা নেই বলে এই রকম
Translated English Text: Like this
Hate Speech Detection Result: [{'label': 'toxic', 'score': 0.00384994107298553}]

Original Bangla Text: বিশ্ব বেহায়া
Translated English Text: The world is pernicious
Hate Speech Det

In [119]:
import pandas as pd


data =  subset_df['sentence']

df = pd.DataFrame(data)

df['english_text'] = df['sentence'].apply(translate_text)
df['hate_speech'] = df['english_text'].apply(lambda x: detect_hate_speech(x)[0])

In [120]:
df

,sentence,english_text,hate_speech
0,তোদের জন্মের দোষ আছে,You have the fault of your birth,"{'label': 'toxic', 'score': 0.4925139546394348}"
1,দুই বউ নিয়ে মজা করছে লুচ্চা মুস্তাক,Lucha Mustak is joking with two wives,"{'label': 'toxic', 'score': 0.00716389762237668}"
2,ওর সমস্ত পরিবারটা নির্লজ্জ,All of his family is shameless,"{'label': 'toxic', 'score': 0.48732465505599976}"
3,এদেরকে পাবনা পাঠাও,Send them to Pabna,"{'label': 'toxic', 'score': 0.022534504532814026}"
4,লজ্জা নেই বলে এই রকম,Like this,"{'label': 'toxic', 'score': 0.00384994107298553}"
5,বিশ্ব বেহায়া,The world is pernicious,"{'label': 'toxic', 'score': 0.003305187216028571}"
6,এরা সফল কারণ এদের লজ্জা শরম বলতে কিছুই নেই,They are successful because they have nothing ...,"{'label': 'toxic', 'score': 0.10095352679491043}"
7,নিলজ্জাহি মহিলা,Niljahi,"{'label': 'toxic', 'score': 0.009757804684340954}"
8,বুড়ো বয়সে প্রেম রতি ধরেছে,The old man holds love rhythm,"{'label': 'toxic', 'score': 0.0007578557706438..."
9,লজ্জা সরম না থাকলে যা হয়,If the shame is not a simple,"{'label': 'toxic', 'score': 0.03198353946208954}"


In [121]:
results = process_dataset(Y)

In [122]:
for i, (Y, results) in enumerate(zip(Y, results)):
    print(f"Original Bangla Text: {Y}")
    print(f"Translated English Text: {translate_text(Y)}")
    print(f"Hate Speech Detection Result: {results}")
    print()

Original Bangla Text: fault birth
Translated English Text: Fault
Hate Speech Detection Result: [{'label': 'toxic', 'score': 0.00878493208438158}]

Original Bangla Text: lucha mustak joking two wives
Translated English Text: Luch Mustak Joking 2 Vibas
Hate Speech Detection Result: [{'label': 'toxic', 'score': 0.001426147879101336}]

Original Bangla Text: family shameless
Translated English Text: Family Sameepe
Hate Speech Detection Result: [{'label': 'toxic', 'score': 0.0022849177476018667}]

Original Bangla Text: send pabna
Translated English Text: Send Pabna
Hate Speech Detection Result: [{'label': 'toxic', 'score': 0.025103526189923286}]

Original Bangla Text: like
Translated English Text: Like
Hate Speech Detection Result: [{'label': 'toxic', 'score': 0.0014612984377890825}]

Original Bangla Text: world pernicious
Translated English Text: World Perius
Hate Speech Detection Result: [{'label': 'toxic', 'score': 0.004984825383871794}]

Original Bangla Text: successful nothing say shame

In [123]:
import pandas as pd


data =  subset_df['sentence_clean']

df1 = pd.DataFrame(data)

df1['english_text'] = df1['sentence_clean']
df1['hate_speech'] = df1['english_text'].apply(lambda x: detect_hate_speech(x)[0])

In [124]:
df1

,sentence_clean,english_text,hate_speech
0,fault birth,fault birth,"{'label': 'toxic', 'score': 0.04050400108098984}"
1,lucha mustak joking two wives,lucha mustak joking two wives,"{'label': 'toxic', 'score': 0.007819253951311111}"
2,family shameless,family shameless,"{'label': 'toxic', 'score': 0.29706916213035583}"
3,send pabna,send pabna,"{'label': 'toxic', 'score': 0.025103526189923286}"
4,like,like,"{'label': 'toxic', 'score': 0.0014612984377890..."
5,world pernicious,world pernicious,"{'label': 'toxic', 'score': 0.002641454339027405}"
6,successful nothing say shame,successful nothing say shame,"{'label': 'toxic', 'score': 0.09689675271511078}"
7,niljahi,niljahi,"{'label': 'toxic', 'score': 0.009757804684340954}"
8,old man holds love rhythm,old man holds love rhythm,"{'label': 'toxic', 'score': 0.0009387352620251..."
9,shame simple,shame simple,"{'label': 'toxic', 'score': 0.09890053421258926}"
